In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont
import arabic_reshaper
from bidi.algorithm import get_display
import re
from docx import Document

In [ ]:
def get_wrapped_text_2(text, font, max_width):
    wrapped_lines = []
    for line in text.split('\n'):
        if not line.strip():
            wrapped_lines.append('')
            continue
        words = line.split()
        current_line = words[0]
        for word in words[1:]:
            test_line = current_line + " " + word
            bbox = font.getbbox(test_line)
            if bbox[2] - bbox[0] <= max_width:
                current_line = test_line
            else:
                wrapped_lines.append(current_line)
                current_line = word
        wrapped_lines.append(current_line)
    return wrapped_lines

In [ ]:
def text_to_image_2(text, font_path, font_size, output_path, max_width=1200, line_spacing=10, extra_line_spacing=20):
    reshaped_text = arabic_reshaper.reshape(text)
    bidi_text = get_display(reshaped_text)

    font = ImageFont.truetype(font_path, font_size)
    wrapped_lines = get_wrapped_text_2(bidi_text, font, max_width)
    

    # Calculate the maximum width of all lines
    max_line_width = max(font.getbbox(line)[2] - font.getbbox(line)[0] for line in wrapped_lines)
    
    # Calculate total height with extra spacing
    total_height = 0
    for i, line in enumerate(wrapped_lines):
        total_height += font_size + line_spacing
        total_height += extra_line_spacing
        if i == 2 or (i > 2 and (i - 2) % 2 == 0):  # Extra space after 3rd line and then every 2 lines
            total_height += extra_line_spacing

    # Create an image with the calculated dimensions
    image_width = max(max_line_width + 80, max_width + 80)
    image = Image.new('RGB', (image_width, total_height + 80), color='white')
    draw = ImageDraw.Draw(image)

    y_position = 40
    for i, line in enumerate(wrapped_lines):
        # Calculate the width of the current line
        line_width = font.getbbox(line)[2] - font.getbbox(line)[0]
        
        # Calculate the x position to center the line
        x_position = (image_width - line_width) // 2

        draw.text((x_position, y_position), line, font=font, fill='black')
        y_position += font_size + line_spacing
        y_position += extra_line_spacing
        if i == 2 or (i > 2 and (i - 2) % 2 == 0):  # Extra space after 3rd line and then every 2 lines
            y_position += extra_line_spacing

    image.save(output_path)

In [ ]:
def process_poem_file(file_path, output_dir, font_path, font_size, fixed_word):
     # Create the main output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Define the text files directory
    text_dir = os.path.join(output_dir, "text_files")

    # Check if the text files directory exists
    text_files_exist = os.path.exists(text_dir)

    if not text_files_exist:
        os.makedirs(text_dir)
        print(f"Created text files directory: {text_dir}")
    else:
        print(f"Text files directory already exists: {text_dir}")

    doc = Document(file_path)
    full_text = [para.text for para in doc.paragraphs]
    
    poems = []
    current_poem = []
    current_piece_number = 0
    
    pattern = re.compile(f"^{re.escape(fixed_word)}\\s*(\\d+)")
    
    for line in full_text:
        match = pattern.match(line)
        if match:
            piece_number = int(match.group(1))
            if current_poem and piece_number > current_piece_number:
                poems.append('\n'.join(current_poem))
                current_poem = []
            current_piece_number = piece_number
        if line.strip():  # Only add non-empty lines
            current_poem.append(line)
    
    if current_poem:
        poems.append('\n'.join(current_poem))
    
    for i, poem in enumerate(poems, 1):
        # Create image file (always)
        image_filename = f"poem_{i:03d}.png"  # Use 3-digit numbering
        image_path = os.path.join(output_dir, image_filename)
        text_to_image_2(poem, font_path, font_size, image_path)
        
        # Create text file (only if the directory didn't exist before)
        if not text_files_exist:
            text_filename = f"poem_{i:03d}.txt"  # Use 3-digit numbering
            text_path = os.path.join(text_dir, text_filename)
            with open(text_path, 'w', encoding='utf-8') as text_file:
                text_file.write(poem)

In [ ]:
# poem
poem_path = "C:/Users/QAZ/Downloads/divan-hafez.docx"
output_path_m = "D:poemiii/"
fixed_word = "غزل"

In [ ]:
font_path_m = "C:/Users/QAZ/AppData/Local/Microsoft/Windows/Fonts/"
font_size = [16,18,20,22,24]
font_names = ["Nazanin.ttf", "BArash.ttf", "BArshia.ttf", "BDavat.ttf", "SBARDIYA.TTF", "Fantezy.ttf", "Jaleh_MRT.ttf", "Smooth Light_MRT.ttf", "Ghalam-2_MRT.TTF", "NasimB.ttf"]

In [ ]:
for font in font_names:
    font_path = font_path_m + font
    print(font_path)
    for size in font_size:
        output_path = output_path_m + font + '-' + str(size)
        process_poem_file(poem_path, output_path, font_path, size, fixed_word)